In [1]:
import numpy as np
import pandas as pd
import os
import io
import re
from unidecode import unidecode

path = r"C:\Users\User\Desktop\football-transfer-tool-main/"
os.chdir(path)
import warnings
warnings.filterwarnings("ignore")

In [19]:
joueurs = pd.read_csv('data/fifa_cleaned.csv', sep=';') # https://data.world/raghav333/fifa-players
joueurs_bad_dob = pd.read_csv('data/players_20.csv', sep=';') # https://www.kaggle.com/stefanoleone992/fifa-20-complete-player-dataset?select=players_20.csv
fut_db = pd.read_csv('data/fut_players.csv', sep=';') # https://www.kaggle.com/stefanoleone992/fifa-20-ultimate-team-players-dataset?select=fut_bin20_players.csv

In [3]:
# On va garder le data set players_20.csv pour les stats car cest  celui le plus a jour, cependant la birth date 
# n'est pas bonne pour pratiquement tous les joueurs

In [4]:
joueurs_bad_dob.drop(columns=['dob'], inplace=True)

In [5]:
stats_joueurs = pd.merge(joueurs[['id', 'birth_date']], joueurs_bad_dob, 
                left_on=['id'], 
                right_on=['sofifa_id'])

In [20]:
print(stats_joueurs.shape)
print(fut_db.shape)

(13885, 109)
(18775, 95)


## Associer les joueurs à leur ligue respective

In [18]:
# séparer le nom du joueur de son prénom
stats_joueurs[['nom1','nom2']]=stats_joueurs['short_name'].str.split(r'[A-Z]\.', expand=True)
stats_joueurs['nom'] = stats_joueurs.apply(lambda x : unidecode(x['nom1'].strip().lower()) if not x['nom1'] == '' else unidecode(x['nom2'].strip().lower()), axis=1)
stats_joueurs.drop(columns=['nom1', 'nom2'], inplace=True)

# nom des clubs en minuscule
stats_joueurs['club'] = stats_joueurs['club'].apply(lambda x: unidecode(x.lower()))

In [110]:
# on ne retient que les colonnes utiles pour la jointure
fut_db_pertinent = fut_db[['player_name','player_extended_name', 'nationality', 'date_of_birth', 'height','weight']][fut_db.club!='Icons']

In [111]:
fut_db_pertinent.head()

,player_name,player_extended_name,nationality,date_of_birth,height,weight
142,Ronaldo,C. Ronaldo dos Santos Aveiro,Portugal,05/02/1985,187,83
143,Chiellini,Giorgio Chiellini,Italy,14/08/1984,187,85
144,Dybala,Paulo Dybala,Argentina,15/11/1993,177,75
145,Sandro,Alex Sandro Lobo Silva,Brazil,26/01/1991,180,80
146,Bonucci,Leonardo Bonucci,Italy,01/05/1987,190,85


In [112]:
# conversion des dates de naissanc en format date identique 
stats_joueurs.birth_date = pd.to_datetime(stats_joueurs.birth_date, format = '%d/%m/%Y')
fut_db_pertinent.date_of_birth = pd.to_datetime(fut_db_pertinent.date_of_birth, format='%d/%m/%Y')

In [113]:
# travail à la main sur quelques différences entre les clés de jointure
# fut_db_pertinent['club'] = fut_db_pertinent['club'].apply(lambda x : x.strip() if not x == 'Piemonte Calcio' else 'Juventus')
fut_db_pertinent['nationality'] = fut_db_pertinent['nationality'].apply(lambda x : x.strip() if not x == 'Holland' else 'Netherlands')
fut_db_pertinent['nationality'] = fut_db_pertinent['nationality'].apply(lambda x : x.strip() if not x == "Côte d'Ivoire" else 'Ivory Coast')
fut_db_pertinent['nationality'] = fut_db_pertinent['nationality'].apply(lambda x : x.strip() if not x == 'Bosnia and Herzegovina' else 'Bosnia Herzegovina')
fut_db_pertinent['player_name'] = fut_db_pertinent['player_name'].apply(lambda x: unidecode(x.lower()))
# fut_db_pertinent['club'] = fut_db_pertinent['club'].apply(lambda x: unidecode(x.lower()))

In [114]:
# jointure entre les deux df, le but étant de récupérer la ligue dans laquelle le joueur joue 
players = pd.merge(stats_joueurs, fut_db_pertinent, 
                left_on=['birth_date','nationality', 'height_cm','weight_kg'], 
                right_on=['date_of_birth', 'nationality', 'height','weight'], how='left')

In [115]:
players.drop_duplicates(inplace=True)

In [116]:
players.shape

(13911, 114)

In [118]:
players.head()

,id,birth_date,sofifa_id,player_url,short_name,long_name,age,height_cm,weight_kg,nationality,...,rb,poste_,poste,note_perf,nom,player_name,player_extended_name,date_of_birth,height,weight
0,158023,1987-06-24,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,32,170,72,Argentina,...,63+2,RW,ATT,93.0,messi,messi,Lionel Messi,1987-06-24,170.0,72.0
7,190460,1992-02-14,190460,https://sofifa.com/player/190460/christian-eri...,C. Eriksen,Christian Dannemann Eriksen,27,181,76,Denmark,...,66+3,RM,ATT,83.0,eriksen,eriksen,Christian Eriksen,1992-02-14,181.0,76.0
11,195864,1993-03-15,195864,https://sofifa.com/player/195864/paul-pogba/20...,P. Pogba,Paul Pogba,26,191,84,France,...,74+3,LDM,MDC,78.0,pogba,pogba,Paul Pogba,1993-03-15,191.0,84.0
12,198219,1991-06-04,198219,https://sofifa.com/player/198219/lorenzo-insig...,L. Insigne,Lorenzo Insigne,28,163,59,Italy,...,58+3,LS,ATT,87.0,insigne,insigne,Lorenzo Insigne,1991-06-04,163.0,59.0
16,201024,1991-06-20,201024,https://sofifa.com/player/201024/kalidou-kouli...,K. Koulibaly,Kalidou Koulibaly,28,187,89,Senegal,...,77+3,LCB,DC,81.0,koulibaly,koulibaly,Kalidou Koulibaly,1991-06-20,187.0,89.0


In [120]:
whosna = players[pd.isna(players.player_name)]
whosna.shape

(1094, 114)

In [121]:
# les joueurs présents dans le dataframe whosna sont les joueurs pour lesquels les transferts ont été effectués avant que le  jeu ne sorte
# par conséquent le club n'est pas le meme entre la table fifa et la table fut (qui permet de retrouver la ligue par rapport au club)

## Creation d'indicateurs de performance

In [122]:
# définir un seul poste par joueur
players['poste_'] = players.apply(lambda x: x['team_position'] if not (x['team_position'] == 'SUB') | (x['team_position'] == 'RES') else x['player_positions'].split(', ')[0], axis=1)

In [123]:
# classer par catégorie les postes
def_cen = ['CB','LCB','RCB']
def_lat = ['RB','RWB','LWB','LB']
mil_def = ['CDM','CM', 'LCM', 'RCM','LDM','RDM']
mil_off = ['CAM', 'RAM', 'LAM']
attaq = ['RS','LS', 'CF', 'ST', 'LF','RF','LW', 'RW', 'LM','RM']

In [124]:
# création de 6 postes : def central, def latéraux, milieu def, milieu off, et attaquant
# dans le but de créer les indicateurs de performance pour chaque poste
players['poste'] = np.where(
    players['poste_'].isin(def_cen), 'DC', 
    np.where(players['poste_'].isin(def_lat), 'LAT',
             np.where(players['poste_'].isin(mil_def), 'MDC',
                     np.where(players['poste_'].isin(mil_off), 'MOC',
                             np.where(players['poste_'].isin(attaq), 'ATT', 'GK')))))

In [125]:
# classer les notes des caractéristiques des joueurs en 'familles' qui caractérisent les joueurs pour un poste spécifique
rigueur = ['defending_marking', 'power_strength', 'mentality_composure', 'mentality_aggression', 'defending_standing_tackle','power_jumping']
recuperation = ['mentality_interceptions', 'defending_sliding_tackle', 'power_stamina']
distribution = ['skill_long_passing', 'mentality_vision']
percussion = ['skill_ball_control', 'skill_dribbling', 'movement_acceleration', 'movement_sprint_speed', 'movement_agility', 'movement_balance']
danger = ['movement_reactions', 'attacking_short_passing', 'attacking_crossing']
finition = ['mentality_composure', 'mentality_positioning', 'skill_curve', 'attacking_finishing', 'attacking_volleys']
gk = ['gk_diving','gk_handling', 'gk_kicking','gk_positioning', 'gk_reflexes', 'movement_reactions']

In [126]:
# calcul de la note de performance

gardien = round(players[gk].mean(axis=1))

dc = round(players[rigueur+distribution+recuperation].mean(axis=1))

lat = round(players[rigueur+distribution+recuperation+percussion+danger+finition].mean(axis=1))

mdc = pd.concat([players[rigueur+distribution+recuperation]*3, players[percussion+danger+finition]], axis=1).sum(axis=1)
perf_mdc = mdc//(3*len(rigueur+distribution+recuperation)+len(percussion+danger+finition))

moc = pd.concat([players[rigueur+distribution+recuperation], players[percussion+danger+finition]*3], axis=1).sum(axis=1)
perf_moc = moc//(len(rigueur+distribution+recuperation)+3*len(percussion+danger+finition))

att = round(players[percussion+danger+finition].mean(axis=1))

# en fonction du poste du joueur associée la note de performance en fonction de ses caractéristiques
players['note_perf'] = np.where(
    players['poste']=='DC', dc, 
    np.where(players['poste'] == 'LAT', lat,
            np.where(players['poste'] == 'MDC', perf_mdc,
                    np.where(players['poste'] == 'MOC', perf_moc,
                            np.where(players['poste'] == 'ATT', att, 
                                    gardien)))))

In [127]:
# les 10 meilleurs joueurs selon leur indice de perf
players[['nom','poste','note_perf']].sort_values('note_perf', ascending=False).head(10)

,nom,poste,note_perf
0,messi,ATT,93.0
15823,hazard,ATT,89.0
15843,cristiano ronaldo,ATT,88.0
50,ter stegen,GK,88.0
58,salah,ATT,88.0
15711,ederson,GK,87.0
12,insigne,ATT,87.0
26,mbappe,ATT,87.0
15804,oblak,GK,87.0
39,neuer,GK,87.0


In [128]:
# regarder les meilleurs joueurs selon leur poste
a = players[['nom','poste','note_perf', 'overall']][players.poste=='GK'].sort_values('note_perf', ascending=False)

In [129]:
a.head(10)

,nom,poste,note_perf,overall
50,ter stegen,GK,88.0,90
39,neuer,GK,87.0,88
15711,ederson,GK,87.0,88
15804,oblak,GK,87.0,91
15732,alisson,GK,87.0,89
15836,de gea,GK,86.0,89
15741,handanovic,GK,84.0,88
105,navas,GK,84.0,87
260,gulacsi,GK,84.0,85
48,courtois,GK,84.0,88


In [130]:
# rapporter la note  sur une base de 5 (pour une note sur 5 étoiles par ex)
group_poste = players['note_perf'].groupby(players['poste'])

def perf_base5(x):
    if x['poste'] == 'GK':
        return (x['note_perf']/group_poste.get_group('GK').max())*5
    elif x['poste'] == 'DC':
        return (x['note_perf']/group_poste.get_group('DC').max())*5
    elif x['poste'] == 'LAT':
        return (x['note_perf']/group_poste.get_group('LAT').max())*5
    elif x['poste'] == 'MDC':
        return (x['note_perf']/group_poste.get_group('MDC').max())*5
    elif x['poste'] == 'MOC':
        return (x['note_perf']/group_poste.get_group('MOC').max())*5
    else:
        return (x['note_perf']/group_poste.get_group('ATT').max())*5

In [131]:
players['perf'] = players.apply(lambda x: perf_base5(x), axis=1)

In [132]:
players[['nom','poste','note_perf', 'overall','perf']].head(10)

,nom,poste,note_perf,overall,perf
0,messi,ATT,93.0,94,5.000000
7,eriksen,ATT,83.0,88,4.462366
11,pogba,MDC,78.0,88,4.756098
12,insigne,ATT,87.0,87,4.677419
16,koulibaly,DC,81.0,89,4.764706
20,van dijk,DC,85.0,90,5.000000
26,mbappe,ATT,87.0,89,4.677419
33,aguero,ATT,86.0,89,4.623656
39,neuer,GK,87.0,88,4.943182
43,cavani,ATT,79.0,88,4.247312


In [133]:
players.head()

,id,birth_date,sofifa_id,player_url,short_name,long_name,age,height_cm,weight_kg,nationality,...,poste_,poste,note_perf,nom,player_name,player_extended_name,date_of_birth,height,weight,perf
0,158023,1987-06-24,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,32,170,72,Argentina,...,RW,ATT,93.0,messi,messi,Lionel Messi,1987-06-24,170.0,72.0,5.000000
7,190460,1992-02-14,190460,https://sofifa.com/player/190460/christian-eri...,C. Eriksen,Christian Dannemann Eriksen,27,181,76,Denmark,...,RM,ATT,83.0,eriksen,eriksen,Christian Eriksen,1992-02-14,181.0,76.0,4.462366
11,195864,1993-03-15,195864,https://sofifa.com/player/195864/paul-pogba/20...,P. Pogba,Paul Pogba,26,191,84,France,...,LDM,MDC,78.0,pogba,pogba,Paul Pogba,1993-03-15,191.0,84.0,4.756098
12,198219,1991-06-04,198219,https://sofifa.com/player/198219/lorenzo-insig...,L. Insigne,Lorenzo Insigne,28,163,59,Italy,...,LS,ATT,87.0,insigne,insigne,Lorenzo Insigne,1991-06-04,163.0,59.0,4.677419
16,201024,1991-06-20,201024,https://sofifa.com/player/201024/kalidou-kouli...,K. Koulibaly,Kalidou Koulibaly,28,187,89,Senegal,...,LCB,DC,81.0,koulibaly,koulibaly,Kalidou Koulibaly,1991-06-20,187.0,89.0,4.764706


In [134]:
players.to_csv('data/players.csv', index=False)

In [3]:
p = pd.read_csv('data/joueurs_avec_stats_fifa.csv')

In [20]:
# https://cdn.sofifa.net/players/212/831/22_240.png
new_url = p['url_img'].apply(lambda x : x.split('.')[2].split('/')[-1][:3]+'/'+x.split('.')[2].split('/')[-1][3:])

In [26]:
p['url_img'] = new_url.apply(lambda x: f"https://cdn.sofifa.net/players/{x}/22_240.png")

In [29]:
p.to_csv('data/joueurs_avec_stats_fifa.csv', index=False)